In [42]:
#This notebook is used to read csv file from kaggle and create database table using pandas and pyspark

!pip install opendatasets
!pip install pandas
!pip install pyspark


In [43]:
import pandas as pd
import opendatasets as od
from pyspark.sql import SparkSession

#downloading the public dataset from Kaggle (refer readme for more steps)
od.download("https://www.kaggle.com/datasets/fatemehmehrparvar/obesity-levels?select=ObesityDataSet_raw_and_data_sinthetic.csv")


Skipping, found downloaded files in "./obesity-levels" (use force=True to force download)


In [44]:
#About the dataset
#Height    -Height
#Weight    -Weight
#FHWO	     -Overweight family members
#FAVC	     -Consume high-calorie foods frequently
#FCVC	     -Number of meals where you usually eat vegetables
#NCP	     -Number of main meals a day
#CAEC	     -Eat food between meals
#SMOKE     -How often you smoke
#CH2O	     -Liters of water you drink a day
#SCC	     -Monitor the calories you consume daily
#FAF	     -Frequency of days per week that you often have physical activity
#TUE	     -Time of use of technological devices on a daily basis
#CALC	     -Frequency of alcohol intake.
#MTRANS    -Means of transportation that you use regularly
#NObeyesdad-Body mass index

#reading csv file using pandas
df = pd.read_csv("/content/obesity-levels/ObesityDataSet_raw_and_data_sinthetic.csv")
df

,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,21.000000,Female,1.620000,64.000000,no,no,2.0,3.0,no,no,2.000000,yes,0.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
1,21.000000,Female,1.520000,56.000000,Sometimes,no,3.0,3.0,yes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,23.000000,Male,1.800000,77.000000,Frequently,no,2.0,3.0,no,no,2.000000,yes,2.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
3,27.000000,Male,1.800000,87.000000,Frequently,no,3.0,3.0,no,no,2.000000,no,2.000000,0.000000,Sometimes,Walking,Overweight_Level_I
4,22.000000,Male,1.780000,89.800000,Sometimes,no,2.0,1.0,no,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,20.976842,Female,1.710730,131.408528,Sometimes,yes,3.0,3.0,no,no,1.728139,yes,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,21.982942,Female,1.748584,133.742943,Sometimes,yes,3.0,3.0,no,no,2.005130,yes,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,22.524036,Female,1.752206,133.689352,Sometimes,yes,3.0,3.0,no,no,2.054193,yes,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,24.361936,Female,1.739450,133.346641,Sometimes,yes,3.0,3.0,no,no,2.852339,yes,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [56]:
#create spark SQL session
spark = SparkSession.builder.getOrCreate()

#create dB table df1 from dataframe df
df1 = spark.createDataFrame(df)
df1
df1.show()
df1.columns

+----+------+------+------+----------+----+----+---+---+-----+----+------------------------------+---+---+----------+--------------------+-------------------+
| Age|Gender|Height|Weight|      CALC|FAVC|FCVC|NCP|SCC|SMOKE|CH2O|family_history_with_overweight|FAF|TUE|      CAEC|              MTRANS|         NObeyesdad|
+----+------+------+------+----------+----+----+---+---+-----+----+------------------------------+---+---+----------+--------------------+-------------------+
|21.0|Female|  1.62|  64.0|        no|  no| 2.0|3.0| no|   no| 2.0|                           yes|0.0|1.0| Sometimes|Public_Transporta...|      Normal_Weight|
|21.0|Female|  1.52|  56.0| Sometimes|  no| 3.0|3.0|yes|  yes| 3.0|                           yes|3.0|0.0| Sometimes|Public_Transporta...|      Normal_Weight|
|23.0|  Male|   1.8|  77.0|Frequently|  no| 2.0|3.0| no|   no| 2.0|                           yes|2.0|1.0| Sometimes|Public_Transporta...|      Normal_Weight|
|27.0|  Male|   1.8|  87.0|Frequently|  no| 3.

['Age',
 'Gender',
 'Height',
 'Weight',
 'CALC',
 'FAVC',
 'FCVC',
 'NCP',
 'SCC',
 'SMOKE',
 'CH2O',
 'family_history_with_overweight',
 'FAF',
 'TUE',
 'CAEC',
 'MTRANS',
 'NObeyesdad']

In [58]:
#create temprory view temp_1
df1.createOrReplaceTempView("temp_1")
spark.sql("SELECT count(*) from temp_1 where age < 25 ").show()

+--------+
|count(1)|
+--------+
|    1353|
+--------+



In [47]:
spark.sql("SELECT a.age,a.gender,a.height,a.weight from temp_1 a where NObeyesdad in ('Normal_Weight')").show()
spark.sql("SELECT count(*) from temp_1 a where NObeyesdad in ('Normal_Weight')").show()

+----+------+------+------+
| age|gender|height|weight|
+----+------+------+------+
|21.0|Female|  1.62|  64.0|
|21.0|Female|  1.52|  56.0|
|23.0|  Male|   1.8|  77.0|
|29.0|  Male|  1.62|  53.0|
|23.0|Female|   1.5|  55.0|
|22.0|  Male|  1.64|  53.0|
|24.0|  Male|  1.78|  64.0|
|22.0|  Male|  1.72|  68.0|
|22.0|  Male|  1.65|  56.0|
|23.0|  Male|  1.77|  60.0|
|22.0|Female|   1.7|  66.0|
|22.0|Female|  1.65|  60.0|
|21.0|  Male|  1.85|  68.0|
|20.0|  Male|   1.6|  50.0|
|21.0|  Male|   1.7|  65.0|
|23.0|Female|   1.6|  52.0|
|19.0|  Male|  1.75|  76.0|
|23.0|  Male|  1.68|  70.0|
|24.0|Female|  1.77|  76.0|
|22.0|  Male|  1.65|  62.0|
+----+------+------+------+
only showing top 20 rows

+--------+
|count(1)|
+--------+
|     287|
+--------+



In [48]:
spark.sql("SELECT a.age,a.gender,a.height,a.weight from temp_1 a where smoke = 'yes' and calc in ('Frequently','Sometimes')").show()

+----+------+------+------+
| age|gender|height|weight|
+----+------+------+------+
|21.0|Female|  1.52|  56.0|
|21.0|  Male|  1.66|  62.0|
|30.0|  Male|  1.76| 112.0|
|19.0|Female|  1.63|  76.0|
|19.0|Female|  1.65|  56.0|
|23.0|  Male|  1.74|  93.5|
|38.0|Female|   1.5|  60.0|
|20.0|  Male|  1.83|  72.0|
|21.0|Female|  1.63|  60.0|
|30.0|  Male|  1.92| 130.0|
|26.0|  Male|  1.91|  84.0|
|23.0|Female|  1.48|  60.0|
|26.0|  Male|  1.75|  80.0|
|23.0|Female|  1.63|  83.0|
|26.0|Female|  1.56| 102.0|
|23.0|Female|   1.6|  78.0|
|21.0|Female|  1.63|  66.0|
|33.0|  Male|  1.85|  99.0|
|20.0|Female|  1.64|  49.0|
|56.0|  Male|  1.79|  90.0|
+----+------+------+------+
only showing top 20 rows



In [49]:
spark.sql("SELECT a.age,a.gender,a.height,a.weight from temp_1 a where smoke = 'yes' and calc in ('Frequently','Sometimes') order by a.age").show()

+----+------+------+------+
| age|gender|height|weight|
+----+------+------+------+
|18.0|  Male|  1.86| 110.0|
|19.0|Female|  1.63|  76.0|
|19.0|Female|  1.65|  56.0|
|20.0|  Male|  1.83|  72.0|
|20.0|Female|  1.64|  49.0|
|21.0|Female|  1.52|  56.0|
|21.0|  Male|  1.66|  62.0|
|21.0|Female|  1.63|  60.0|
|21.0|Female|  1.63|  66.0|
|21.0|  Male|  1.82|  67.0|
|21.0|  Male|  1.75|  62.0|
|23.0|  Male|  1.74|  93.5|
|23.0|Female|  1.48|  60.0|
|23.0|Female|  1.63|  83.0|
|23.0|Female|   1.6|  78.0|
|23.0|  Male|  1.75| 120.0|
|25.0|  Male|  1.66|  68.0|
|26.0|  Male|  1.91|  84.0|
|26.0|  Male|  1.75|  80.0|
|26.0|Female|  1.56| 102.0|
+----+------+------+------+
only showing top 20 rows



In [50]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'yes' and a.calc in ('Frequently','Sometimes')").show()

+--------+
|count(1)|
+--------+
|      38|
+--------+



In [51]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'yes' and a.calc not in ('Frequently','Sometimes')").show()

+--------+
|count(1)|
+--------+
|       6|
+--------+



In [52]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'no' and a.calc in ('Frequently','Sometimes')").show()

+--------+
|count(1)|
+--------+
|    1433|
+--------+



In [53]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'no' and a.calc not in ('Frequently','Sometimes')").show()

+--------+
|count(1)|
+--------+
|     634|
+--------+



In [54]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'yes' and a.calc in ('Frequently','Sometimes') and gender = 'Female'").show()

+--------+
|count(1)|
+--------+
|      11|
+--------+



In [55]:
spark.sql("SELECT count(*) from temp_1 a where a.smoke = 'yes' and a.calc in ('Frequently','Sometimes') and gender = 'Male'").show()

+--------+
|count(1)|
+--------+
|      27|
+--------+

